In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#**0. Thư viện**

##Cài đặt 

In [ ]:
# !pip3 numpy torc sklearn matplotlib pandas

##Import

In [ ]:
# Add Module Path - To Import Custom Modules
# Modules Path
ModulePath = "/content/drive/My Drive/Study/KLTN/Google Colab/0.0 Python Modules/"

import sys
sys.path.append(ModulePath)

In [ ]:
import numpy as np
import pandas as pd
import torch as th
from torch.autograd import Variable as V
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix

# Libs for Keras
from keras.models import load_model


# Own Custom Module to import models and constants
from models import *
from constants import *

import matplotlib.pyplot as plt
import math
import os
from datetime import date
import timeit

#**1. Chuẩn bị**

##Paths

In [ ]:
# Base Path
base_path = "/content/drive/My Drive/Study/KLTN/"
Dataset_Path = base_path + "Dataset/NSL-KDD Processed/Final - For Using/"
SavedModelPath = base_path + "Saved Model/"
Generated_Dataset_Save_Path = base_path + "Generated Datasets/"

# Dataset Path
Trainsets_Path = Dataset_Path + 'Trainset/'
g_trainset_path = Trainsets_Path + "GAN-G.csv"
d_trainset_path = Trainsets_Path + "GAN-D.csv"
testset_path = Dataset_Path + "Testset/" + "KDDTest+.csv"


# GAN Saved Models Paths
GAN_Model_Path = SavedModelPath + 'GANModel/'

# IDS Pytorch Saved Models Paths
IDS_Model_Path = SavedModelPath + 'IDSModel/'

##Global Variables

In [ ]:
# Global Variables
N_FEATURES = 41
# IDS
IDS_INPUT_DIM = N_FEATURES
IDS_OUTPUT_DIM = 2
# ATTACK_CATEGORIES = ['DOS', 'U2R_AND_R2L']

POS_FUNCTIONAL_FEATURES = {'DOS': DOS_FEATURES, 'U2R_AND_R2L': U2R_AND_R2L_FEATURES}
POS_NONFUNCTIONAL_FEATURES = {}
for attack_category, pos_functional_feature in POS_FUNCTIONAL_FEATURES.items():
    pos_nonfunctional_feature = []
    for i in range(N_FEATURES):
        if i not in pos_functional_feature:
            pos_nonfunctional_feature.append(i)
    POS_NONFUNCTIONAL_FEATURES[attack_category] = pos_nonfunctional_feature

IDS_MODELS = {'MLP': MLP, 'CNN': None, 'RNN': None}

In [ ]:
print('Position of Functional Features\n  ', POS_FUNCTIONAL_FEATURES)
print('Position of Functional Features:\n  ', POS_NONFUNCTIONAL_FEATURES)

Position of Functional Features
   {'DOS': [0, 1, 2, 3, 4, 5, 6, 7, 8, 23, 24, 25, 26, 27, 28, 29, 30], 'U2R_AND_R2L': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]}
Position of Functional Features:
   {'DOS': [9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40], 'U2R_AND_R2L': [23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40]}


# **2. Định nghĩa Model**

##Models

In [ ]:
# Load from model.py module

##Functions

### Processing Data Functions

In [ ]:
# Processing Data Functions
# create_batch2 - Hàm tạo Batch
def create_batch2(x,batch_size):
    a = list(range(len(x)))
    np.random.shuffle(a)
    x = x[a]
    batch_x = [x[batch_size * i : (i+1)*batch_size,:] for i in range(len(x)//batch_size)]
    return np.array(batch_x)
    
# preprocess_malicious_data - Hàm tiền xử lý dữ liệu tấn công
def preprocess_malicious_data(dataset, attack_category):
    if attack_category != 'DOS' and attack_category != 'U2R_AND_R2L':
      raise ValueError("Preprocess Data Fail: Invalid Attack Category")
    attack_data = dataset[dataset['class'] == attack_category]
    del attack_data["class"]
    return np.array(attack_data)

### IDS Functions

In [ ]:
# IDS Functions
# get_keras_ids_path - Get path for Keras IDS Models
def get_keras_ids_path(model_name, attack_category, created_date):
    if attack_category != 'DOS' and attack_category != 'U2R_AND_R2L':
        raise ValueError("Preprocess Data Fail: Invalid Attack Category")

    ids_path = str(f"{IDS_Model_Path}{attack_category}/Deep_Learning/created_date_{created_date}_{model_name}.h5")

    if not os.path.exists(ids_path):
        raise ValueError(f"Invalid path: {ids_path}\nNot exist file!")
    return ids_path

# load_keras_ids_model - Load Keras IDS Models
def load_keras_ids_model(model_name, attack_category, created_date):
    ids_model_path = get_keras_ids_path(model_name, attack_category, created_date)
    ids_model = load_model(ids_model_path)
    print(f"{4*' '}Loaded IDS Model From: {ids_model_path}")
    return ids_model

### GAN Functions

In [ ]:
# GAN Functions
# init_generator - Khoi tao model GAN-G
def init_generator(input_dim, output_dim, adversarial_attack_type = 1):
    if adversarial_attack_type == 1:
        generator = Generator_A1(input_dim, output_dim)
    elif adversarial_attack_type == 2:
        generator = Generator_A2(input_dim, output_dim)
    elif adversarial_attack_type == 3:
        '''
            adversarial_attack_type 3: G-input is nonfunctional feature, G-out is adversarial nonfuncitonal feature.
            --> input_dim = output_dim = len(nonfunctional_features)
        ''' 
        generator = Generator_A2(output_dim, output_dim)
    elif adversarial_attack_type == 4:
        '''
            adversarial_attack_type 4: G-input in random noise nf-dim (num. of nonfunctional features)
            --> input_dim = output_dim = len(nonfunctional_features)
        '''
        generator = Generator_A1(output_dim, output_dim)
    else:
        raise ValueError("Init GAN - Generator: Invalid Adversarial Attack Type")
    return generator

# gen_adversarial_attack - Tao luu luong tan cong doi khang
def gen_adversarial_attack(generator, noise_dim, raw_attack, attack_category, adversarial_attack_type = 1):
    if adversarial_attack_type == 1 or adversarial_attack_type == 4:
        adversarial_attack = generator(noise_dim, raw_attack, attack_category, POS_NONFUNCTIONAL_FEATURES)
    elif adversarial_attack_type == 2:
        batch_size = len(raw_attack)
        noise = V(th.Tensor(np.random.uniform(0,1,(batch_size, noise_dim))))
        generator_out = generator(noise)
        adversarial_attack = gen_adversarial_attack_a2(generator_out, raw_attack, attack_category, POS_NONFUNCTIONAL_FEATURES)
    elif  adversarial_attack_type == 3:
        pos_nonfunctional_feature = POS_NONFUNCTIONAL_FEATURES[attack_category]
        # get nonfunctional features val
        nonfunctional_feature = raw_attack[:, pos_nonfunctional_feature]
        generator_out = generator(nonfunctional_feature)
        adversarial_attack = gen_adversarial_attack_a2(generator_out, raw_attack, attack_category, POS_NONFUNCTIONAL_FEATURES)
    else:
        raise ValueError("Init GAN - Generator: Invalid Adversarial Attack Type")
    return adversarial_attack

# train_generator - Train Generator
def train_generator(generator, discriminator, optimizer_G, noise_dim, attack_traffic, attack_category, adversarial_attack_type):
    for p in discriminator.parameters():  
        p.requires_grad = False
    optimizer_G.zero_grad()        
    # GAN-G Generate Adversarial Attack
    adversarial_attack = gen_adversarial_attack(generator, noise_dim, attack_traffic, attack_category, adversarial_attack_type)
    # GAN-D predict, GAN-G update parameter
    D_pred = discriminator(adversarial_attack)
    g_loss = -th.mean(D_pred)
    g_loss.backward()
    optimizer_G.step()
    return g_loss

# train_discriminator - Train Discriminator
def train_discriminator(discriminator, ids_model, generator, critic_iters, clamp, optimizer_D, normal_b, noise_dim, attack_traffic, attack_category, adversarial_attack_type):
    run_d_loss = 0
    cnt = 0
    for p in discriminator.parameters(): 
        p.requires_grad = True
    for c in range(critic_iters):
        optimizer_D.zero_grad()
        for p in discriminator.parameters():
            p.data.clamp_(-clamp, clamp)
        # GAN-G Generate Adversarial Attack
        adversarial_attack = gen_adversarial_attack(generator, noise_dim, attack_traffic, attack_category, adversarial_attack_type)
        # Make data to feed IDS
        ids_input = th.cat((adversarial_attack,normal_b))
        l = list(range(len(ids_input)))
        np.random.shuffle(l)
        ids_input = V(th.Tensor(ids_input[l]))
        ids_input_keras = ids_input.reshape(ids_input.shape[0], N_FEATURES, 1)      # Keras Format
        # IDS Predict
        ids_pred = V(th.Tensor(ids_model.predict(ids_input_keras)))
        ids_pred_label = th.argmax(nn.Sigmoid()(ids_pred),dim = 1).detach().numpy()
        pred_normal = ids_input.numpy()[ids_pred_label==0]
        pred_attack = ids_input.numpy()[ids_pred_label==1]
        if len(pred_attack) == 0:
            cnt += 1
            break
        # Make GAN-D input
        D_noraml = discriminator(V(th.Tensor(pred_normal)))
        D_attack= discriminator(V(th.Tensor(pred_attack)))
        # Loss and Update Parameter
        loss_normal = th.mean(D_noraml)
        loss_attack = th.mean(D_attack)
        if GAN_variant == 'WGAN':
            d_loss = loss_attack - loss_normal #+ LAMBDA * gradient_penalty
        elif GAN_variant == 'WGAN-GP':
            gradient_penalty = compute_gradient_penalty(discriminator, normal_b.data, adversarial_attack.data)
            d_loss = loss_attack - loss_normal + LAMBDA * gradient_penalty
        d_loss.backward()
        optimizer_D.step()
        run_d_loss += d_loss.item()
    return run_d_loss, cnt

# compute_gradient_penalty - Compute Gradient Penalty
def compute_gradient_penalty(D, normal_t, attack_t):
    alpha = th.Tensor(np.random.random((normal_t.shape[0], 1)))
    between_n_a = (alpha * normal_t + ((1 - alpha) * attack_t)).requires_grad_(True)
    d_between_n_a = D(between_n_a)
    adv = V(th.Tensor(normal_t.shape[0], 1).fill_(1.0), requires_grad=False)

    gradients = autograd.grad(
        outputs=d_between_n_a,
        inputs=between_n_a,
        grad_outputs=adv,
        create_graph=True,
        retain_graph=True,
        only_inputs=True,
    )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

### Generate Adversarial Attack Functions

In [ ]:
# cal_dr - Tinh DR
def cal_dr(ids_model, normal, raw_attack, adversarial_attack):
    # Make data to feed IDS contain: Attack & Normal
    o_ids_input = th.cat((raw_attack, normal))
    a_ids_input = th.cat((adversarial_attack,normal))
    # Shuffle Input
    l = list(range(len(a_ids_input)))
    np.random.shuffle(l)
    o_ids_input = o_ids_input[l]
    o_ids_input = o_ids_input.reshape(o_ids_input.shape[0], N_FEATURES, 1)      # Keras Format
    o_ids_input = np.array(o_ids_input)
    a_ids_input = a_ids_input[l].detach()
    a_ids_input = a_ids_input.reshape(a_ids_input.shape[0], N_FEATURES, 1)      # Keras Format
    a_ids_input = np.array(a_ids_input)
    # IDS Predict Label
    o_ids_pred = th.Tensor(ids_model.predict(o_ids_input))
    o_pred_label = th.argmax(nn.Sigmoid()(o_ids_pred),dim = 1).detach().numpy()
    a_ids_pred = th.Tensor(ids_model.predict(a_ids_input))
    a_pred_label = th.argmax(nn.Sigmoid()(a_ids_pred),dim = 1).detach().numpy()
    # True Label
    ids_true_label = np.r_[np.ones(BATCH_SIZE),np.zeros(BATCH_SIZE)][l]
    # Calc DR
    tn1, fn1, fp1, tp1 = confusion_matrix(ids_true_label,o_pred_label).ravel()
    tn2, fn2, fp2, tp2 = confusion_matrix(ids_true_label,a_pred_label).ravel()
    origin_dr = tp1/(tp1 + fp1)
    adversarial_dr = tp2/(tp2 + fp2)
    return origin_dr, adversarial_dr

###Save Generated Dataset

In [ ]:
# save_generated_dataset - Save Generated Dataset
def save_generated_dataset(df, path):
    df.to_csv(path, index = False, header=True)
    print(f"	    Generated Dataset Saved\n	    Saved Path: {path}")

# **3. Gen&Save Dataset**

In [ ]:
#@title **OPTION**
# ids_dl_model_name - ids_dl_model_name
ids_dl_model_name = "CNN" #@param ["CNN", "RNN_1"]

# ids_created_date - ids_created_date
IDS_Model_Created_Auto = {'CNN': '2020-07-17',
                          'RNN_1': '2020-07-22'
                          }
ids_created_date = 'Auto' #@param ['Auto', '2020-07-17']
if ids_created_date == 'Auto':
    ids_created_date = IDS_Model_Created_Auto[ids_dl_model_name]
    print(f"IDS: {ids_dl_model_name} - created on: \t{ids_created_date}")

# # GAN_variant - GAN_variant
# GAN_variant = 'WGAN' #@param ['WGAN', 'WGANGP']

# MAX_EPOCH - MAX_EPOCH
MAX_EPOCH = 100 #@param [100, 10, 2, 1] {type:"raw"}
ATTACK_CATEGORIES = 'ALL' #@param ['ALL', 'DOS', 'U2R_AND_R2L']

# ATTACK_CATEGORIES - ATTACK_CATEGORIES
if ATTACK_CATEGORIES == 'ALL':
    ATTACK_CATEGORIES = ['DOS', 'U2R_AND_R2L']
else:
    ATTACK_CATEGORIES = [ATTACK_CATEGORIES]
print(f"ATTACK_CATEGORIES: \t\t{ATTACK_CATEGORIES}")

# ADVERSARIAL_ATTACK_TYPES - ADVERSARIAL_ATTACK_TYPES
ADVERSARIAL_ATTACK_TYPES = 1 #@param [1, 3, 4] {type:"raw"}
if ADVERSARIAL_ATTACK_TYPES == 0:
    ADVERSARIAL_ATTACK_TYPES = [1, 3, 4]
else:
    ADVERSARIAL_ATTACK_TYPES = [ADVERSARIAL_ATTACK_TYPES]
print(f"ADVERSARIAL_ATTACK_TYPES: \t{ADVERSARIAL_ATTACK_TYPES}")

# NOISE_DIM - NOISE_DIM
NOISE_DIM = 9 #@param[9, 41] {type:"raw"}

# gan_model_time_created - gan_model_time_created
default_gan_model_time_created = {'CNN': '2020-07-22',
                                  'RNN_1': '2020-07-22'
                                  }
gan_model_time_created = "Auto" #@param ["Auto", "Today"] {allow-input: true}
if gan_model_time_created == 'Auto':
    gan_model_time_created = default_gan_model_time_created[ids_dl_model_name]
elif gan_model_time_created == 'Today':
    gan_model_time_created = str(date.today())
print(f"GAN Model Created Time: \t{gan_model_time_created}")

# BATCH_SIZE - BATCH_SIZE
BATCH_SIZE = 256 #@param ["256"] {type:"raw", allow-input: true}

IDS: CNN - created on: 	2020-07-17
ATTACK_CATEGORIES: 		['DOS', 'U2R_AND_R2L']
ADVERSARIAL_ATTACK_TYPES: 	[1]
GAN Model Created Time: 	2020-07-22


## **RUN**

In [ ]:
# Load Testset
testset = pd.read_csv(testset_path)
DATASET_COLUMNS = testset.columns[0:41]
print(f"Amount of KDDTest+: \t\t{len(testset)}")
# test_normal
test_normal = np.array(testset[testset["class"] == 'Normal'])[:,:-1]
# Create batch of normal traffic
test_batch_normal = create_batch2(test_normal,BATCH_SIZE).astype('float64')
print(f"Amount of Normal:\t\t{len(test_normal)} ({len(test_batch_normal)} batchs - {BATCH_SIZE} records/batch)")

Amount of KDDTest+: 		20123
Amount of Normal:		9711 (37 batchs - 256 records/batch)


In [ ]:
# Adversarial Traffic Evaluating
print(f"{40*'='} GEN & SAVE DATASET {40*'='}")
print(f"{80*'='}\n IDS Model: {ids_dl_model_name}")

for adversarial_attack_type in ADVERSARIAL_ATTACK_TYPES:
    print(f"{80*'*'}\n Adversarial Attack Type : {adversarial_attack_type}")
    for attack_category in ATTACK_CATEGORIES:
        print(f"{80*'-'}\n - Attack Category: {attack_category}")
        
        # Load Keras IDS Model
        ids_model = load_keras_ids_model(ids_dl_model_name, attack_category, ids_created_date)
        # Init GAN-G model
        G_OUTPUT_DIM = len(POS_NONFUNCTIONAL_FEATURES[attack_category])     # Generator output is number of nonfunctional feature
        print(f"    nf              : {G_OUTPUT_DIM} (num. of nonfunctional features)")
        if adversarial_attack_type == 4 or adversarial_attack_type == 3:
            NOISE_DIM = G_OUTPUT_DIM                                        # Adversarial Attack Type 3,4: Noise dim = nonfunctional feature
        G_INPUT_DIM = NOISE_DIM                                             # G_INPUT_DIM = NOISE_DIM
        print(f"    GAN-G NOISE DIM : {NOISE_DIM}")
        print(f"    GAN-G INPUT DIM : {G_INPUT_DIM}")
        print(f"    GAN-G OUTPUT DIM: {G_OUTPUT_DIM}")
        generator = init_generator(G_INPUT_DIM,G_OUTPUT_DIM, adversarial_attack_type)
        # Load Attack Dataset
        test_raw_attack = preprocess_malicious_data(testset, attack_category)
        # Create batch of attack traffic
        batch_attack = create_batch2(test_raw_attack, BATCH_SIZE)
        n_batch_attack = len(batch_attack)
        print(f"{4*' '}Amout of {attack_category}:\t{len(test_raw_attack)} ({n_batch_attack} batchs - {BATCH_SIZE} records/batch)")
        
        # Calc DR through each epoch
        gan_g_folder_path = str(f"{GAN_Model_Path}Deep_Learning/{ids_dl_model_name}/{attack_category}/{adversarial_attack_type}/")
        print(f"{4*' '}GAN Models Folder: {gan_g_folder_path}")
        for epoch in range(0, MAX_EPOCH + 1, 10):
            # Load GAN-G Model
            model_g_save_name = f"time_created_{gan_model_time_created}_GAN_G_{1 if epoch == 0 else epoch}epoch.pth"
            gan_g_model_path = gan_g_folder_path + model_g_save_name
            if epoch == 0 and not os.path.isfile(gan_g_model_path):
                continue
            param = th.load(gan_g_model_path,map_location=lambda x,y:x)
            generator.load_state_dict(param)

            generator.eval()

            generated_dataset = pd.DataFrame(columns=DATASET_COLUMNS)
            o_dr,a_dr =[],[]
            with th.no_grad():
                for idx, bn in enumerate(test_batch_normal):
                    normal_b = th.Tensor(bn)
                    attack_b = th.Tensor(batch_attack[idx % n_batch_attack])
                    # Generate Adversarial Traffic
                    adversarial_attack_b = gen_adversarial_attack(generator, NOISE_DIM, attack_b, attack_category, adversarial_attack_type)
                    if (idx < n_batch_attack):
                        epoch_dataset = pd.DataFrame(data = np.array(adversarial_attack_b), columns=DATASET_COLUMNS)
                        generated_dataset = generated_dataset.append(epoch_dataset, ignore_index = True)
                    

                    # Calc DR
                    origin_dr, adversarial_dr = cal_dr(ids_model, normal_b, attack_b, adversarial_attack_b)
                    o_dr.append(origin_dr)
                    a_dr.append(adversarial_dr)
            eir = 1 - (np.mean(a_dr)/np.mean(o_dr))
            print(f"\t {epoch:3d} epochs:\tOrigin DR : {np.mean(o_dr)*100:.2f}% \t Adversarial DR : {np.mean(a_dr)*100:.2f}% \t EIR : {eir*100:.2f}%")
            
            generated_dataset_folder_path = str(f"{Generated_Dataset_Save_Path}Deep_Learning/{ids_dl_model_name}/{attack_category}/{adversarial_attack_type}/")
            if not os.path.exists(generated_dataset_folder_path):
                os.makedirs(generated_dataset_folder_path)
            generated_dataset_file_path = generated_dataset_folder_path + str(f"time_created_{date.today()}_{epoch}epoch.csv")
            save_generated_dataset(generated_dataset, generated_dataset_file_path)

======================================== GEN & SAVE DATASET ========================================
 IDS Model: CNN
********************************************************************************
 Adversarial Attack Type : 1
--------------------------------------------------------------------------------
 - Attack Category: DOS
    Loaded IDS Model From: /content/drive/My Drive/Study/KLTN/Saved Model/IDSModel/DOS/Deep_Learning/created_date_2020-07-17_CNN.h5
    nf              : 24 (num. of nonfunctional features)
    GAN-G NOISE DIM : 9
    GAN-G INPUT DIM : 9
    GAN-G OUTPUT DIM: 24
    Amout of DOS:	7460 (29 batchs - 256 records/batch)
    GAN Models Folder: /content/drive/My Drive/Study/KLTN/Saved Model/GANModel/Deep_Learning/CNN/DOS/1/
	   0 epochs:	Origin DR : 81.83% 	 Adversarial DR : 0.65% 	 EIR : 99.20%
	    Generated Dataset Saved
	    Saved Path: /content/drive/My Drive/Study/KLTN/Generated Datasets/Deep_Learning/CNN/DOS/1/time_created_2020-08-05_0epoch.csv
	  10 epochs:	

# **4. Compare Datasets**

In [ ]:
#@title **OPTION**
# ATTACK_CATEGORIES - ATTACK_CATEGORIES
ATTACK_CATEGORIES = 'ALL' #@param ['ALL', 'DOS', 'U2R_AND_R2L']
if ATTACK_CATEGORIES == 'ALL':
    ATTACK_CATEGORIES = ['DOS', 'U2R_AND_R2L']
else:
    ATTACK_CATEGORIES = [ATTACK_CATEGORIES]
print(f"ATTACK_CATEGORIES: \t\t{ATTACK_CATEGORIES}")

# dataset_date_created - dataset_date_created
dataset_date_created = "Today" #@param ["Today", "2020-08-01"] {allow-input: true}
if dataset_date_created == "Today":
    dataset_date_created = str(date.today())
print(f"dataset_date_created: \t\t{dataset_date_created}")

# EPOCHS - EPOCHS
EPOCHS =  100#@param ["100"] {type:"raw", allow-input: true}
if type(EPOCHS) == int:
    EPOCHS = [EPOCHS]
print(f"EPOCHS: \t\t\t{EPOCHS}")

ATTACK_CATEGORIES: 		['DOS', 'U2R_AND_R2L']
dataset_date_created: 		2020-08-05
EPOCHS: 			[100]


##**RUN**

In [ ]:
# get_cat_vals - Get unique category value of a dataset
def get_cat_vals(dataset, attack_category):
    cat_num, cat_vals = [], []
    for nonfunctional_feature in  POS_NONFUNCTIONAL_FEATURES[attack_category]:
        np_cat_val = dataset[:, nonfunctional_feature]
        cat_val = pd.Series(np_cat_val)
        cat_num.append(len(cat_val.unique()))
        cat_vals.append(cat_val.unique())
    return cat_num, cat_vals

# count_different_element - count number of element of adversarials that not in originals
def count_different_element(adversarials_set, originals_set):
    count_set = []
    for idx, adversarials in enumerate(adversarials_set):
        originals = originals_set[idx]
        count = 0
        for adversarial in adversarials:
            if adversarial not in originals:
                count+= 1
        count_set.append(count)
    return count_set

In [ ]:
# Pandas Display Setting
pd.set_option("display.max_rows", None, "display.max_columns", None)

# Load dataset
testset = pd.read_csv(testset_path)
DATASET_COLUMNS = testset.columns[0:41]

print(f"<{40*'-'} COMPARE DATASET {40*'-'}>")
for adversarial_attack_type in ADVERSARIAL_ATTACK_TYPES:
    print(f"{80*'*'}\n Adversarial Attack Type : {adversarial_attack_type}")
    for attack_category in ATTACK_CATEGORIES:
        print(f"<{80*'='}>\n  # Attack Category: {attack_category}")
        
        # compare_table - init compare_table: Create Column Nonfunctional Feature
        data = {'Nonfunctional_Feature':DATASET_COLUMNS[POS_NONFUNCTIONAL_FEATURES[attack_category]]}
        compare_table = pd.DataFrame(data)

        # compare_table - compare_table add column Origin Number of Category and Origin Categories
        test_raw_attack = preprocess_malicious_data(testset, attack_category)
        cat_num, cat_vals = get_cat_vals(test_raw_attack, attack_category)
        compare_table['Origin-Num_Cat'] = cat_num
        compare_table['Origin-Cat'] = cat_vals

        for epoch in EPOCHS:
            print(f"    --> Computing in Epoch: {epoch}...")
            # Load Adversarial Attack Dataset
            adversarial_set_path = str(f"{Generated_Dataset_Save_Path}Deep_Learning/{ids_dl_model_name}/{attack_category}/{adversarial_attack_type}/time_created_{dataset_date_created}_{epoch}epoch.csv")
            adversarial_set = pd.read_csv(adversarial_set_path)
            adversarial_attack = np.array(adversarial_set)
            print(f"         Adversarial Attack Data Shape: {adversarial_attack.shape}")
            
            # compare_table - compare_table add column Adversarial Categories of each Epoch
            ad_cat_num, ad_cat_vals = get_cat_vals(adversarial_attack, attack_category)
            compare_table[str(f'E{epoch}-Num_Ad_Cat')] = ad_cat_num
            compare_table[str(f'E{epoch}-Ad_Cat')] = ad_cat_vals
            compare_table[str(f'E{epoch}-Num_NotIn_Origin')] = count_different_element(ad_cat_vals, cat_vals)
        print_columns = []
        print_columns.append('Nonfunctional_Feature')
        print_columns.append('Origin-Num_Cat')
        for e in EPOCHS:
            print_columns.append(str(f"E{epoch}-Num_Ad_Cat"))
            print_columns.append(str(f"E{epoch}-Num_NotIn_Origin"))
        
        # Print the Statistic
        print(20*'-')
        print(compare_table[print_columns].to_markdown())
        
        for epoch in EPOCHS:
            not_in_column = str(f"E{epoch}-Num_NotIn_Origin")
            print(f"\n ==> E{epoch}: Num. of feature not exist before: {len(compare_table[not_in_column][compare_table[not_in_column] != 0])}/{len(compare_table[not_in_column])}")

            # Save the Statistic
            folder_save_path = os.path.dirname(adversarial_set_path)
            statistic_file_name = str(f'Statistic_of_E{epoch}_dataset_{dataset_date_created}.csv')
            compare_table.to_csv(folder_save_path +'/'+ statistic_file_name)
            print(f" The full statistic saved at: {folder_save_path}")
            print(f" Name: {statistic_file_name}\n\n")

<---------------------------------------- COMPARE DATASET ---------------------------------------->
********************************************************************************
 Adversarial Attack Type : 1
<================================================================================>
  # Attack Category: DOS
    --> Computing in Epoch: 100...
         Adversarial Attack Data Shape: (7424, 41)
--------------------
|    | Nonfunctional_Feature       |   Origin-Num_Cat |   E100-Num_Ad_Cat |   E100-Num_NotIn_Origin |
|---:|:----------------------------|-----------------:|------------------:|------------------------:|
|  0 | hot                         |                4 |              7377 |                    7377 |
|  1 | num_failed_logins           |                1 |                 1 |                       0 |
|  2 | logged_in                   |                2 |              7256 |                    7256 |
|  3 | num_compromised             |                2 |          

The Saved Model stored in [Google Drive - GAN Model](https://drive.google.com/drive/u/1/folders/1VNFW-k5SbR0eGsJ_np3U-W3Rcz_n4I8N)

The Result of Code stored in [Github - Thesis](https://github.com/thetinybug/thesis-IDSGAN)